<div dir="rtl" style="text-align: center">

<b><h1> پروژه‌ی استخراج دستور غذا از متن </h1></b>
<h2> تمرین 1 </h2>
<h2> درس بازیابی پیشرفته اطلاعات </h2>
<h3> جناب آقای دکتر عسگری </h3>
<h3> اعضای گروه </h3>
پارسا محمدیان - 98102284
<br/>
سارا آذرنوش - 98170668
<br/>
کهبد آئینی - 98101209 
<br/>
<br/>
دانشگاه صنعتی شریف
<br/>
دانشکده مهندسی کامپیوتر
<hr/>
</div>

<div dir="rtl">
<h2>
افزودن کتابخانه های مورد نیاز
</h2>
در این قسمت کتابخانه‌های مورد نیاز 
<code>import</code>
شده‌اند. توجه شود که تمامی کتابخانه‌ها 
built-in 
زبان پایتون هستند به جز کتابخانه 
<code>parsi_io</code> 
که باید به صورت دستی در مسیری که برای پایتون قابل یافتن باشد اضافه شود.
</div>

In [229]:
import re
import time
import string
import json

from parsi_io.modules.number_extractor import NumberExtractor


<div dir="rtl">
<h2>
کلاس 
<code>Span</code>
</h2>
از این کلاس برای ذخیره بازه استفاده شده است. همچنین این کلاس توابع کمکی در مورد بازه‌ها نیز در اختیارمان قرار می‌دهد.
</div>

In [230]:
class Span:
    def __init__(self, s, e):
        self.s = s
        self.e = e

    def get_tuple(self) -> tuple:
        return (self.s, self.e)

    def get_list(self) -> list:
        return [self.s, self.e]

    def from_re_span(re_span):
        return Span(re_span[0], re_span[1])

    def seperate_text(self, text: str) -> str:
        return text[self.s:self.e]


<div dir="rtl">
<h2>
استخراج دستور غذا از متن
</h2>
در این قسمت یک کلاس به نام 
<code>RecipeExtractor</code>
داریم که تمام کارهای مربوط به استخراج دستور غذا را انجام می‌دهد. این کلاس به این صورت کار می‌کند که کاربر یک شی از روی کلاس با استفاده از رشته ورودی می‌سازد، سپس تابع 
<code>process</code>
را برای این شی اجرا می‌کند. در این تابع خروجی‌های موردنیاز کاربر در فیلدهای شی ذخیره می‌شود و کاربر می‌تواند آن‌ها را بخواند. نحوه کار این تابع به این صورت است که ابتدا تیتر دستور پخت حذف می‌شود تا در استخراج مواد اولیه مشکلی ایجاد نکند. البته طول رشته حذف شده را ذخیره می‌کنیم تا در تعیین 
<code>Span</code>
اشتباه نکنیم. سپس برای هر ماده اولیه 
<code>Span</code>
مشخص می‌کنیم. پس از آن به کمک همان 
<code>Span</code>ها 
و تابع تشخیص عدد در کتابخانه 
<code>parsi_io</code>
نوع ماده اولیه و مقدار آن را مشخص می‌کنیم. سپس سراغ دستور پخت می‌رویم و بازه آن را بدست می‌آوریم و سپس از روی بازه متن را بدست می‌آوریم. در مرحله بعد تمام اعداد موجود در 
<code>Span</code>
را با مقداری که طول تیتر حذف شده بود، جمع می‌کنیم. در مرحله آخر علائم نگارشی را از مواد اولیه حذف می‌کنیم و 
whitespaceهای 
اضافی را از متن خروجی نهایی حذف می‌کنیم تا خروجی خواناتری داشته باشیم.
</div>

In [231]:
class RecipeExtractor:

    INGREDIENTS_TITLE_REGEX = re.compile(r'((?!مواد)[\n|[^\n]])*مواد([^\n]*)(\n+\s*\n*)')
    SINGLE_INGREDIENT_REGEX = re.compile(r'[^\n]+(?P<newline>\n+)')
    RECIPE_TITLE_REGEX = re.compile(r'(طرز|نحوه|نحوه‌ی|نحوه ی|دستور|شیوه|روش) (تهیه|تهیه‌ی|تهیه|پخت|آماده‌سازی|آماده سازی)([^\n]*)(\n+)')
    ARABIC_NUMBERS_REGEX = re.compile(r'ربع|ثلث|به میزان|به مقدار|خمس|دنگ|نصف')

    def __init__(self, text: str):
        self.num_extractor = NumberExtractor()
        self.text = text
        self.title_length = 0
        self.span_ingredients = []
        self.ingredients = []
        self.quantities = []
        self.span_recipe = None
        self.recipe = None

    def process(self) -> None:
        self.__ingredient_title_remover()
        self.__process_span_ingredients()
        self.__process_ingredients_and_quantities()
        self.__process_recipe()
        self.__relocate_spans()
        self.__post_process_texts()

    def __ingredient_title_remover(self) -> None:
        s = re.search(self.INGREDIENTS_TITLE_REGEX, self.text)
        print(s)
        self.text = self.text[s.span()[1]:]
        self.title_length = s.span()[1]
        return self.text

    def __process_span_ingredients(self) -> None:
        s = re.search(RecipeExtractor.RECIPE_TITLE_REGEX, self.text)
        self.span_recipe = Span(s.span()[1], len(self.text))
        all_string = self.text[:s.span()[0]]
        f = re.finditer(RecipeExtractor.SINGLE_INGREDIENT_REGEX, all_string)
        for ingredient in f:
            span = Span.from_re_span(ingredient.span())
            span.e -= len(ingredient.group('newline'))
            self.span_ingredients.append(span)

    def __process_ingredients_and_quantities(self) -> None:
        for span_ingredient in self.span_ingredients:
            ingredient_string = span_ingredient.seperate_text(self.text)
            number_span = self.num_extractor.run(ingredient_string)
            if len(number_span) != 0:
                self.ingredients.append(
                    ingredient_string[:number_span[0]['span'][0]])
                self.quantities.append(
                    ingredient_string[number_span[0]['span'][0]:])
            else:
                m = re.search(
                    RecipeExtractor.ARABIC_NUMBERS_REGEX, ingredient_string)
                if not m is None:
                    self.ingredients.append(ingredient_string[:m.span()[0]])
                    self.quantities.append(ingredient_string[m.span()[0]:])
                else:
                    self.ingredients.append(ingredient_string)
                    self.quantities.append('')

    def __process_recipe(self) -> None:
        self.recipe = self.span_recipe.seperate_text(self.text)

    def __relocate_spans(self) -> None:
        for i in range(len(self.span_ingredients)):
            self.span_ingredients[i].s += self.title_length
            self.span_ingredients[i].e += self.title_length
        self.span_recipe.s += self.title_length
        self.span_recipe.e += self.title_length

    def __post_process_texts(self) -> None:
        for c in string.punctuation:
            for i in range(len(self.ingredients)):
                self.ingredients[i] = self.ingredients[i].replace(c, '')
                self.quantities[i] = self.quantities[i].replace(c, '')
        for i in range(len(self.ingredients)):
            self.ingredients[i] = self.ingredients[i].strip()
            self.quantities[i] = self.quantities[i].strip()
        self.recipe = self.recipe.strip()
        self.recipe = self.recipe.replace('\n', ' ')


<div dir="rtl">
<h2>
تابع اولیه
</h2>
تابع اولیه که باتوجه به دستور کار ورودی گرفته و با صدا زدن تابع توصیف شده در قسمت قبلی، خروجی مورد نیاز را تولید می‌کند.
</div>

In [232]:
def run(input: str) -> dict:
    start = time.time()
    output = {}
    extractor = RecipeExtractor(input)
    extractor.process()
    output['ingredients'] = extractor.ingredients
    output['quantity'] = extractor.quantities
    output['recipe'] = extractor.recipe
    output['span_ingredients'] = [s.get_list() for s in extractor.span_ingredients]
    output['span_recipe'] = extractor.span_recipe.get_list()
    end = time.time()
    output['time'] = end-start
    print(extractor.title_length)
    return output


<div dir="rtl">
<h2>
مثال
</h2>
</div>

In [233]:
inp = """مواد لازم:
اردک درسته ۱ عدد
پیاز ۱ عدد
انار ۳ قاشق غذاخوری
رب گوجه ۱ قاشق غذاخوری
رب نارنج ۲ ق غ
نمک و فلفل و ادویه
سبزی معطر
طرز تهیه:
اردک را بعد از شستشو و تمیز کردن با ادویه و نمک و فلفل مزه دار می‌کنیم؛ و پیاز را طلایی کرده و به آن زردچوبه اضافه می‌کنیم سپس انار و سبزی را اضافه کرده و خیلی کم تفت می‌دهیم و در آخر ادویه را اضافه می‌کنیم. بعد از خنک شدن مواد داخل شکم اردک را از مواد پر کرده و با خلال دندان شگم اردک را می‌بندیم و در تابه‌ای آن را سرخ می‌کنیم. سپس در قابلمه‌ای که اندازه اردک باشه رب انار و گوجه را با همراه نصف لیوان آب می گذاریم تا ملایم بپزد."""

In [234]:
print(json.dumps(run(inp), ensure_ascii=False, indent=4))


<re.Match object; span=(0, 11), match='مواد لازم:\n'>
11
{
    "ingredients": [
        "اردک درسته",
        "پیاز",
        "انار",
        "رب گوجه",
        "رب نارنج",
        "نمک و فلفل و ادویه",
        "سبزی معطر"
    ],
    "quantity": [
        "۱ عدد",
        "۱ عدد",
        "۳ قاشق غذاخوری",
        "۱ قاشق غذاخوری",
        "۲ ق غ",
        "",
        ""
    ],
    "recipe": "اردک را بعد از شستشو و تمیز کردن با ادویه و نمک و فلفل مزه دار می‌کنیم؛ و پیاز را طلایی کرده و به آن زردچوبه اضافه می‌کنیم سپس انار و سبزی را اضافه کرده و خیلی کم تفت می‌دهیم و در آخر ادویه را اضافه می‌کنیم. بعد از خنک شدن مواد داخل شکم اردک را از مواد پر کرده و با خلال دندان شگم اردک را می‌بندیم و در تابه‌ای آن را سرخ می‌کنیم. سپس در قابلمه‌ای که اندازه اردک باشه رب انار و گوجه را با همراه نصف لیوان آب می گذاریم تا ملایم بپزد.",
    "span_ingredients": [
        [
            11,
            27
        ],
        [
            28,
            38
        ],
        [
            39,
            58

In [235]:
inp = """دستور شماره یک 

مواد لازم:

 بادمجان ۴عدد
 پیاز یک عدد
 لوبیا چشم بلبلی یک پیمانه
 آب تمبر هندی نصف لیوان
 نمک، فلفل و زردچوبه به میزان لازم
 آب یک لیوان
 روغن به میزان لازم

طرز تهیه:

لوبیا چشم بلبلی را بپزید. در حین پخت دو بار آب آن را عوض کنید تا خاصیت نفاخ بودن حبوبات از آن گرفته شود. حالا در یک تابه، پیاز را خرد کرده و حرارت دهید تا وقتی که سبک شود. در این زمان بادمجان‌ها را پوست کنده و در آب نمک بگذارید تا تلخی آن از بین برود. سپس آنها را شسته و به صورت مکعبی خرد کنید. بادمجان را به پیازداغ اضافه کنید و به آن نمک، فلفل، زردچوبه بزنید. اجازه دهید بادمجان‌ها کمی در روغن تفت بخورند. سپس یک لیوان آب به مواد اضافه کرده و به مدت نیم ساعت حرارت دهید تا بادمجان‌ها نرم شوند. نصف یک بسته کوچک تمبر هندی را در نصف لیوان آب خیس کنید و بعد از نیم ساعت از صافی رد کنید. لوبیا چشم بلبلی را روی بادمجان‌ها ریخته و آب تمبر هندی را هم اضافه کنید. دقت کنید که چون تمبر هندی شور است، در اضافه کردن نمک غذا کمی احتیاط به خرج دهید. ترجیحا نمک را در آخر پخت اضافه کنید. اگر ذائقه تان می‌پذیرد، غذا را تند کنید. این غذا در اصل باید تند و پرادویه باشد. بعد از آن که تمبر هندی به خورد مواد رفت و آب غذا کشیده شد، آن را در ظرف ریخته و با سبزیجات تازه تزئین کنید.
 """

In [236]:
print(json.dumps(run(inp), ensure_ascii=False, indent=4))


<re.Match object; span=(17, 30), match='مواد لازم:\n\n '>
30
{
    "ingredients": [
        "بادمجان",
        "پیاز",
        "لوبیا چشم بلبلی",
        "آب تمبر هندی",
        "نمک، فلفل و زردچوبه",
        "آب",
        "روغن"
    ],
    "quantity": [
        "۴عدد",
        "یک عدد",
        "یک پیمانه",
        "نصف لیوان",
        "به میزان لازم",
        "یک لیوان",
        "به میزان لازم"
    ],
    "recipe": "لوبیا چشم بلبلی را بپزید. در حین پخت دو بار آب آن را عوض کنید تا خاصیت نفاخ بودن حبوبات از آن گرفته شود. حالا در یک تابه، پیاز را خرد کرده و حرارت دهید تا وقتی که سبک شود. در این زمان بادمجان‌ها را پوست کنده و در آب نمک بگذارید تا تلخی آن از بین برود. سپس آنها را شسته و به صورت مکعبی خرد کنید. بادمجان را به پیازداغ اضافه کنید و به آن نمک، فلفل، زردچوبه بزنید. اجازه دهید بادمجان‌ها کمی در روغن تفت بخورند. سپس یک لیوان آب به مواد اضافه کرده و به مدت نیم ساعت حرارت دهید تا بادمجان‌ها نرم شوند. نصف یک بسته کوچک تمبر هندی را در نصف لیوان آب خیس کنید و بعد از نیم ساعت از صافی ر